# Linear Program/network project by Paige McKenzie

Includes code to perform analysis discussed in my [blog post]().

Good reference source: my previous [work](https://github.com/p-mckenzie/example-projects/blob/925661d96675048d7faed1ddcb89ca41b708d149/Edmunds%20analysis.ipynb).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

import networkx as nx

In [ ]:
pairs = pd.read_csv("Snowshoe_data.csv")
pairs.head()

In [ ]:
edges = pairs.apply(lambda row:(row['Start'], row['End'], {'Name':row['Name'], 'Type':row['Type']}), axis=1).values

In [ ]:
MG = nx.MultiGraph()

In [ ]:
keys = MG.add_edges_from(edges)

In [ ]:
# get the positions of each item in the network
positions = pd.DataFrame(nx.spring_layout(MG, seed=1), index=['x','y']).T

# initialize color column for edge data
pairs['color'] = pairs['Type'].str.lower().replace('lift', 'grey')

In [ ]:
from bokeh.plotting import Figure, show, output_notebook#, output_file
from bokeh.models import GraphRenderer, StaticLayoutProvider, ColumnDataSource, MultiLine, Circle, LabelSet
output_notebook()


fig = Figure(title="Test", x_range=(-1.1,1.1), y_range=(-1.1,1.1),
              tools="", toolbar_location=None)

# plot the nodes
fig.circle(x='x', y='y', size=15, source=ColumnDataSource(positions))

# make the graph
graph = GraphRenderer()

# intialize the nodes
graph.node_renderer.glyph = Circle(size=15, fill_color='blue')
graph.node_renderer.data_source.data = dict(
    index=positions.index.values)

# plot the edges
graph.edge_renderer.glyph = MultiLine(line_color="color", line_width=5)
graph.edge_renderer.data_source.data = dict(
    start=pairs['Start'].values,
    end=pairs['End'].values,
    color=pairs['color'].values)

# tell the graph where the nodes are
source = dict(zip(positions.index.values, zip(positions['x'].values, positions['y'].values)))

graph_layout = source
graph.layout_provider = StaticLayoutProvider(graph_layout=graph_layout)

fig.renderers.append(graph)

# add labels to each node
labels = LabelSet(x='x', y='y', text='index', text_align='right', x_offset=-10, y_offset=10,
              text_color='black', level='glyph',
              source=ColumnDataSource(positions), text_font_size='8pt')

fig.add_layout(labels)



show(fig)